In [ ]:
%pylab inline

# Классификация

Задача классификации - одна из задач машинного обучения, имея признаковое описание объекта нужно определить, представителем какой группы он является.

[Задача и данные для семинара](https://www.kaggle.com/iabhishekofficial/mobile-price-classification)

Подробное описание задачи приводится на странице по вышеуказанной ссылке

Как правило, цена телефона зависит от его характеристик. В данном случае предлагается предсказать не точную цену телефона а его ценовую категорию

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = pd.read_csv("train.csv")

In [ ]:
data.head()

In [ ]:
data.price_range.unique()

Для упрощения начальной задачи бинаризуем целевой признак. Сейчас там четыре класса, мы же упростим и будем предсказывать принадлежность телефона к классу "подороже"

In [ ]:
data['price_range_bin'] = [1 if x > 1 else 0 for x in data.price_range]
data.head()

Для начала выделим из наших данных целевой признак

In [ ]:
y = data.price_range_bin.values
X = data[data.columns[:-2]].values
X.shape, y.shape

Для того, чтобы понять, как хорошо мы обучаем предсказательную модель, нам нужна размеченная выборка, на которой мы будем проверять.
Выборка, на которой мы учимся, не подходит для этих целей. Так как знания об этих объектах уже будут лежать в нашей модели, а в реальной жизни будут встречаться и новые, ранее неизвестные объекты.

ВОПРОС: откуда нам взять размеченную выборку?

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=2018)

print (X_train.shape, y_train.shape)
print (X_test.shape, y_test.shape)

### Метод ближайшего соседа

Самый простой способ предсказать класс объекта - это найти наиболее схожий с ним объект и приписать его к этому же классу.

ВОПРОС: как определить схожесть между объектами в векторном представлении?

ВОПРОС со звезочкой: почему евклидова метрика будет не очень хороша?

$\text{cosine_similarity} = \cos(\theta) = {a \cdot b \over \|a\| \|b\|} = \frac{ \sum\limits_{i=1}^{n}{a_i \times b_i} }{ \sqrt{\sum\limits_{i=1}^{n}{(a_i)^2}} \times \sqrt{\sum\limits_{i=1}^{n}{(a_i)^2}} }$

In [ ]:
class NearestNeighbourPredictor(object):
    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train
        self.X_train_norm = np.linalg.norm(X_train, axis=1)
    
    def predict(self, X):
        X_norm = np.linalg.norm(X, axis=1)
        similarity = X.dot(self.X_train.T)
        similarity = similarity / np.repeat(X_norm, len(self.X_train)).reshape((len(X), len(self.X_train)))
        similarity = similarity / np.repeat(self.X_train_norm, len(X)).reshape((len(self.X_train), len(X))).T
        most_similar = np.argmax(similarity, axis=1)
        return np.array([y_train[x] for x in most_similar])

In [ ]:
nnPredictor = NearestNeighbourPredictor()
nnPredictor.fit(X_train, y_train)
y_predicted = nnPredictor.predict(X_test)

Данный подход - очень простая модель. у нее практически нет параметров (только метрика сходства).

Предсказывая принадлежность телефона к сегменту дорогих, мы можем совершить ошибки двух типов
* Ошибка первого рода (_type I errors, $\alpha$ errors, false positives_): ложное срабатывание предсказательной модели, в нашем случае телефон отнесен к числу дорогих, будучи дешевым
* Ошибка второго рода (_type II errors, $\beta$ errors, false negatives_): модель не выявила дорогой телефон, отнеся его к дешевым

ВОПРОС: Какая из этих ошибок критичнее? Что если речь идет о медицнском обследовании?

ВОПРОС: Как нам оценить качество предсказывающей модели?

Метрик на самом деле придумать можно очень много. Остановимся на основных

![](http://blog.exsilio.com/wp-content/uploads/2016/09/table-blog.png)

Точность (precision): $P = \frac{TP}{TP + FP}$

Полнота (Recall): $R = \frac{TP}{TP + FN}$

True Positive Rate: $TPR = \frac{TP}{TP + FN}$

False Positive Rate: $FPR = \frac{FP}{FP + TN}$

F-measure: $F = \frac{1}{\alpha\frac{1}{P} + (1 - \alpha)\frac{1}{R}}$

Попробуйте реализовать их самостоятельно:

In [ ]:
def precision(y_real, y_predicted):
    pass
    
def recall(y_real, y_predicted):
    pass

def FMeasure(y_real, y_predicted, alpha = 0.5):
    pass

def truePositiveRate(y_real, y_predicted):
    pass
    
def falsePositiveRate(y_real, y_predicted):
    pass

In [ ]:
print 'P =', precision(y_test, y_predicted)
print 'R =', recall(y_test, y_predicted)
print 'F =', FMeasure(y_test, y_predicted)
print 'TPR =', truePositiveRate(y_test, y_predicted)
print 'FPR =', falsePositiveRate(y_test, y_predicted)


Неплохо для очень простой модели.
Но можно лучше.
Есть предложения?

### Метод ближайших соседей

А что, если взять не один ближайший объект а несколько?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knc_7 = KNeighborsClassifier(n_neighbors=7)
knc_7.fit(X_train, y_train)
y_predicted = knc_7.predict(X_test)
print 'P =', precision(y_test, y_predicted)
print 'R =', recall(y_test, y_predicted)
print 'F =', FMeasure(y_test, y_predicted)
print 'TPR =', truePositiveRate(y_test, y_predicted)
print 'FPR =', falsePositiveRate(y_test, y_predicted)


Уже лучше. 
Более того, теперь мы можем говорить с какой вероятностью объект принадлежит классу:

In [ ]:
y_proba = knc_7.predict_proba(X_test)
y_proba[:10]

В данном случае вероятности - это доля соседей "проголосовавших" за соответствующий класс

Теперь, имея вероятностное представление того, что телефон дорогой, можно усложнить модель введя отсечение по этому значению.  

In [ ]:
y_predicted = (y_proba[:,1] >= 6./7).astype(int)
y_predicted[:10]

ВОПРОС: Как это скажется на метриках?

ВОПРОС: Что произойдет в крайних случаях?

In [ ]:
print 'P =', precision(y_test, y_predicted)
print 'R =', recall(y_test, y_predicted)
print 'F =', FMeasure(y_test, y_predicted)
print 'TPR =', truePositiveRate(y_test, y_predicted)
print 'FPR =', falsePositiveRate(y_test, y_predicted)


Давайте посмотрим, как ведут себя показатели $TPR$ и $FPR$ в зависимости от этого порога

In [ ]:
tpr = []
fpr = []
for i in xrange(8): #вопрос на понимание языка: Почему 8?
    threshold = i / 7.
    y_predicted = (y_proba[:,1] >= threshold).astype(int)
    tpr.append(truePositiveRate(y_test, y_predicted))
    fpr.append(falsePositiveRate(y_test, y_predicted))
plt.plot(range(8), tpr)
plt.plot(range(8), fpr)
plt.legend(['truePositiveRate', 'FalsePostiveRate'])

 Давайте теперь изобразим $FPR$ по одной оси координат, а $TPR$ по другой.
 Полученная кривая называется $ROC$-кривой ($receiver operating characteristic$ - рабочая характеристика приёмника, термин пришедший из радио).

In [ ]:
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

Измеримую интерпретацию $ROC$-кривой дает метрика AUC (area under curve - площадь под кривой).
Чем он выше, тем качественнее классификатор. 
Значение $AUC = 0.5$ говорит о непригодности модели, это равносильно случайному гаданию.

ВОПРОС: о чем говорит значение $AUC < 0.5$?

In [ ]:
from sklearn import metrics
print 'AUC =', metrics.roc_auc_score(y_test, y_proba[:,1])

Как еще можно модифицировать метод ближайших соседей?

ОТВЕТ: 

### Другие подходы к классификации

Можно ли выявить дорогой телефон, не обращаясь к прямому сравнению другими?

Например, дорогой телефон будет иметь более мощный процессор, больше экран, разнообразнее функционал.
Отвечая на эти вопросы получим некоторую древовидную структуру


![](https://photos-3.dropbox.com/t/2/AADDUyO52w2UI3UwMrwiTANuAWjHsWbwNnRN14qDoYRfHg/12/85562713/png/32x32/1/_/1/2/Untitled%20Diagram.png/EMSCr8QFGAEgBygH/Dx-vsv--a1bQ2gq6NcSGsDqcVNnJy6w1dWfdKqJ4TTI?preserve_transparency=1&size=32x32&size_mode=5)

Данный подход называется так и называется - decision tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dTree = DecisionTreeClassifier()
dTree.fit(X_train, y_train)
y_predicted = dTree.predict(X_test)
y_proba = dTree.predict_proba(X_test)

In [ ]:

print 'P =', precision(y_test, y_predicted)
print 'R =', recall(y_test, y_predicted)
print 'F =', FMeasure(y_test, y_predicted)
print 'TPR =', truePositiveRate(y_test, y_predicted)
print 'FPR =', falsePositiveRate(y_test, y_predicted)
print 'AUC =', metrics.roc_auc_score(y_test, y_proba[:,1])
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba[:,1])
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

Для решающего дерева мы также можем выяснить важность критерия для принятия решения

In [ ]:
featureImportance = pd.DataFrame({"feature": data.columns[:-2], "importance": dTree.feature_importances_})
featureImportance.sort(["importance"],ascending=False)

Как видим, мощность процессора не сыграла роли, зато очень критичным оказался объем оперативной памяти.
Так же можно заметить, что 3g, wifi, touchscreen и прочие функции, имеющиеся во всех смартфонах, не играют роли

Много деревьев - это лес. Аналогично есть методы машинного обучения, использующие большое количество деревьеви зовутся они лесами


In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
y_predicted = rfc.predict(X_test)
y_proba = rfc.predict_proba(X_test)

In [ ]:

print 'P =', precision(y_test, y_predicted)
print 'R =', recall(y_test, y_predicted)
print 'F =', FMeasure(y_test, y_predicted)
print 'TPR =', truePositiveRate(y_test, y_predicted)
print 'FPR =', falsePositiveRate(y_test, y_predicted)
print 'AUC =', metrics.roc_auc_score(y_test, y_proba[:,1])
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_proba[:,1])
plt.plot(fpr, tpr)
plt.xlabel('FPR')
plt.ylabel('TPR')

### Подбор гиперпараметров

Давайте посмотрим, как меняется качество моделей, в зависимости от гиперпараметров: Числа деревьев в лесу и количества соседей в методе ближайших соседей.
Начнем с метода kNN

In [ ]:
from collections import defaultdict

In [ ]:
n_neighbours = range(1,10)
n_neighbours.extend(range(10,100, 5))
n_neighbours.extend(range(100,251, 10))
resultsKnn = defaultdict(list)
for n in n_neighbours:
    knc = KNeighborsClassifier(n_neighbors=n)
    knc.fit(X_train, y_train)
    
    y_proba = knc.predict_proba(X_test)
    y_predicted = knc.predict(X_test)
    resultsKnn['AUC'].append(metrics.roc_auc_score(y_test, y_proba[:,1]))
    resultsKnn['n'].append(n)
    resultsKnn['P'].append(metrics.precision_score(y_test, y_predicted))
    resultsKnn['R'].append(metrics.recall_score(y_test, y_predicted))
    resultsKnn['F'].append(metrics.f1_score(y_test, y_predicted))
    
resultsKnn = pd.DataFrame(resultsKnn)
resultsKnn[['n','AUC', 'P', 'R', 'F']]

In [ ]:
plt.plot(n_neighbours, resultsKnn.AUC)
plt.plot(n_neighbours, resultsKnn.P)
plt.plot(n_neighbours, resultsKnn.R)
plt.plot(n_neighbours, resultsKnn.F)
plt.legend(['AUC', 'P', 'R', 'F'])

что же мы видим?
С некоторого значения количества соседей качество моделей начинает падать.
Данный эффект называется _переобучением_: модель избыточной сложности хорошо запоминает тестовую выборку, но на новых, неизвестных данных предсказательная способность падает

Что же с лесами? Лес - это ансамбль очень простых моделей (как правило, используемые деревья очень неглубокие), поэтому по параметру число деревьев эта модель не склонна к переобучению

In [ ]:
n_trees = range(1,10)
n_trees.extend(range(10,100, 5))
n_trees.extend(range(100,251, 10))
resultsForest = defaultdict(list)
for n in n_trees:
    rfc = RandomForestClassifier(n_estimators=n)
    rfc.fit(X_train, y_train)
    y_proba = rfc.predict_proba(X_test)
    y_train_proba= rfc.predict_proba(X_train)
    y_predicted = rfc.predict(X_test)
    resultsForest['AUC'].append(metrics.roc_auc_score(y_test, y_proba[:,1]))
    resultsForest['n'].append(n)
    resultsForest['P'].append(metrics.precision_score(y_test, y_predicted))
    resultsForest['R'].append(metrics.recall_score(y_test, y_predicted))
    resultsForest['F'].append(metrics.f1_score(y_test, y_predicted))
    
resultsForest = pd.DataFrame(resultsForest)
resultsForest[['n','AUC', 'P', 'R', 'F']]

In [ ]:
plt.plot(n_trees, resultsForest.AUC)
plt.plot(n_trees, resultsForest.P)
plt.plot(n_trees, resultsForest.R)
plt.plot(n_trees, resultsForest.F)
plt.legend(['AUC', 'P', 'R', 'F'])

как же подбирать параметры?
Для Этих целей используют _кросс-валидацию_.
Данные разбиваются несколько раз на обучающую и тестовую выборку.
Затем при переборе параметров идет оценка модели на каждой тестовой выборке, результаты агрегируются в одно число, по которому потом идет максимизация качества

в пакете sklearn реализована кросс-валидация по решетке параметров

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
parameters = {'n_neighbors':range(3, 11, 2), 'weights':['uniform', 'distance']}
knc = KNeighborsClassifier()
clf = GridSearchCV(knc, parameters)
clf.fit(X_train, y_train)
clf.best_estimator_

ИНДИВИДУАЛЬНОЕ ЗАДАНИЕ: 

найдите параметры метода ближайших соседей и метода случайного леса, оптимизирующие разные метрики.


см: http://scikit-learn.org/stable/modules/model_evaluation.html#scoring